# Scanning igmspec and qpq for spectra

In [48]:
# imports
import pdb
from collections import OrderedDict

from astropy.coordinates import SkyCoord
from astropy.table import Table

from specdb.specdb import IgmSpec
from specdb import specdb as sdbsdb
from specdb.cat_utils import flags_to_groups

## Load TPE + spectral datasets

In [3]:
tpe_file = 'TPE_DR12_31.2_spec.fits'

In [4]:
reload(sdbsdb)
igmsp = sdbsdb.IgmSpec()
qpq_file = os.getenv('DROPBOX_DIR')+'/QSOPairs/spectra/qpq_oir_spec.hdf5'
qpq = sdbsdb.IgmSpec(db_file=qpq_file, skip_test=True)
# Load TPE table
tpe = Table.read(tpe_file)
b_coords = SkyCoord(ra=tpe['BG_RA'], dec=tpe['BG_DEC'], unit='deg')

Database is igmspec
Created on 2017-Jan-05
Database is QPQ
Created on 2017-Jan-02


## Query catalogs

### igmspec

In [10]:
igm_cat_match, igm_cat, igm_ID = igmsp.qcat.query_coords(b_coords)

Your search yielded 195 matches from 196 input coordinates


In [11]:
tpe[~igm_cat_match]

FG_Z,FG_LOGLV,FG_G_UV,FG_RA,FG_DEC,BG_Z,BG_RA,BG_DEC,BG_LYA_INSTRUMENT,BG_LYA_FILE
float64,float32,float64,float64,float64,float64,float64,float64,str8,str39
1.73456704617,31.3287,2145.93843303,221.122497067,31.2225650079,1.79499995708,221.11651431,31.2205412644,LRIS,SDSSJ144427.97+311313.8_b1200_F.fits.gz


### QPQ

In [12]:
qpq_cat_match, qpq_cat, qpq_ID = qpq.qcat.query_coords(b_coords)

Your search yielded 15 matches from 196 input coordinates


In [15]:
qpq_cat[qpq_cat_match][0:3]

flag_group,sig_zem,flag_zem,RA,DEC,STYPE,zem,QPQ_ID
int64,float64,str8,float64,float64,str3,float64,int64
2,0.0,QPQ,4.557875,14.4154444444,QSO,4.23,2330
64,0.0,QPQ,17.9558333333,14.0377222222,QSO,2.927,2552
64,0.0,BOSS_PCA,24.1307916667,15.0384444444,QSO,3.3882958889,2744


In [17]:
qpq_gnames = flags_to_groups(qpq_cat['flag_group'][qpq_cat_match], qpq.group_dict)
qpq_gnames

array([u'ESI', u'LRIS', u'LRIS', u'SDSS', u'ESI', u'LRIS,SDSS,NIRI',
       u'SDSS', u'SDSS', u'BOSS', u'LRIS', u'SDSS', u'BOSS', u'ESI',
       u'BOSS', u'LRIS,MAGE'], 
      dtype='<U14')

## Generate lists of meta tables

In [14]:
igm_meta_match, igm_meta_list = igmsp.meta_from_coords(b_coords, first=False)

Your search yielded 195 matches from 196 input coordinates
Final query yielded 195 matches.


In [15]:
qpq_meta_match, qpq_meta_list = qpq.meta_from_coords(b_coords, first=False)

Your search yielded 15 matches from 196 input coordinates
Final query yielded 15 matches.


## Instrument Priority

In [81]:
# Instrument, disperser pairs in reverse order of preference;  '' means any disperser is allowed
instr_prior = OrderedDict()
instr_prior['SDSS'] = ''
instr_prior['BOSS'] = ''
instr_prior['LRISb'] = '1200/3400'
instr_prior['MagE'] = ''
instr_prior['ESI'] = 'ECH'
instr_prior['UVES'] = ''
instr_prior['HIRES'] = ''
instr_prior

OrderedDict([('SDSS', ''),
             ('BOSS', ''),
             ('LRISb', '1200/3400'),
             ('MagE', ''),
             ('ESI', 'ECH'),
             ('UVES', ''),
             ('HIRES', '')])

## Loop on Sources to build dbase,group,instr,grating + restrict on Lya

In [88]:
def add_to_specmeta(dbase, meta, qq, spec_meta, zabs, buff=50.):
    for ss,row in enumerate(meta):
        wv_lya = (1+zabs) * 1215.67
        if np.any([(row['WV_MIN'] > wv_lya-buff),(row['WV_MAX'] < wv_lya+buff)]):
            continue
        # 
        if spec_meta[qq]['nspec'] > 0:
            spec_meta[qq]['specm'] += ';'
        spec_meta[qq]['specm'] += ','.join([dbase,row['GROUP'], row['INSTR'], row['DISPERSER']])
        spec_meta[qq]['nspec'] += 1
        # Priority
        try:
            aok = instr_prior[row['INSTR']] in row['DISPERSER']
        except KeyError:
            print('Instr = {:s} not in Priority dict'.format(row['INSTR']))
        else:
            if aok:  # Better choice?
                spec_meta[qq]['nok'] += 1
                pri = instr_prior.keys().index(row['INSTR'])
                if pri > spec_meta[qq]['ibest']:
                    spec_meta[qq]['best_row'] = ss
                    spec_meta[qq]['ibest'] = pri
                    spec_meta[qq]['best_spec'] = ','.join([dbase,row['GROUP'], row['INSTR'], row['DISPERSER']])

In [89]:
# Build
spec_dict = dict(specm='', best_spec='', nspec=0, ibest=-1, nok=0, best_row=-1)
spec_meta = [spec_dict.copy() for i in range(len(tpe))]
# 
for qq,pair in enumerate(tpe):
    # igmspec
    if igm_meta_match[qq]:
        # Grab all meta
        meta = igm_meta_list[qq]
        # Add
        add_to_specmeta('igmsp', meta, qq, spec_meta, pair['FG_Z'])
    # QPQ
    if qpq_meta_match[qq]:
        # Meta + add
        add_to_specmeta('qpq', qpq_meta_list[qq], qq, spec_meta, pair['FG_Z'])

Instr = 2dF not in Priority dict
Instr = 2dF not in Priority dict
Instr = 2dF not in Priority dict
Instr = 2dF not in Priority dict
Instr = GMOS-N not in Priority dict
Instr = 2dF not in Priority dict
Instr = 2dF not in Priority dict
Instr = 2dF not in Priority dict
Instr = 2dF not in Priority dict
Instr = 2dF not in Priority dict
Instr = 2dF not in Priority dict
Instr = 2dF not in Priority dict


In [90]:
spec_meta[0:5]

[{'best_row': 0,
  'best_spec': 'igmsp,BOSS_DR12,BOSS,BOTH',
  'ibest': 1,
  'nok': 2,
  'nspec': 2,
  'specm': 'igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH'},
 {'best_row': 0,
  'best_spec': 'qpq,ESI,ESI,ECH',
  'ibest': 4,
  'nok': 3,
  'nspec': 3,
  'specm': 'igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH;qpq,ESI,ESI,ECH'},
 {'best_row': 0,
  'best_spec': 'igmsp,BOSS_DR12,BOSS,BOTH',
  'ibest': 1,
  'nok': 2,
  'nspec': 2,
  'specm': 'igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH'},
 {'best_row': 0,
  'best_spec': 'igmsp,BOSS_DR12,BOSS,BOTH',
  'ibest': 1,
  'nok': 2,
  'nspec': 2,
  'specm': 'igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH'},
 {'best_row': 0,
  'best_spec': 'igmsp,BOSS_DR12,BOSS,BOTH',
  'ibest': 1,
  'nok': 2,
  'nspec': 2,
  'specm': 'igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH'}]

## Convert to Table

In [91]:
spec_tbl = Table()
for key in spec_dict.keys():
    clm = [sdict[key] for sdict in spec_meta]
    spec_tbl[key] = clm
#
spec_tbl

specm,best_spec,nok,ibest,best_row,nspec
str182,str27,int64,int64,int64,int64
"igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH","igmsp,BOSS_DR12,BOSS,BOTH",2,1,0,2
"igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH;qpq,ESI,ESI,ECH","qpq,ESI,ESI,ECH",3,4,0,3
"igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH","igmsp,BOSS_DR12,BOSS,BOTH",2,1,0,2
"igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH","igmsp,BOSS_DR12,BOSS,BOTH",2,1,0,2
"igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH","igmsp,BOSS_DR12,BOSS,BOTH",2,1,0,2
"igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH","igmsp,BOSS_DR12,BOSS,BOTH",2,1,0,2
"igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH;qpq,LRIS,LRISb,1200/3400","qpq,LRIS,LRISb,1200/3400",3,2,0,3
"igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH","igmsp,BOSS_DR12,BOSS,BOTH",2,1,0,2
"igmsp,BOSS_DR12,BOSS,BOTH;igmsp,SDSS_DR7,SDSS,BOTH","igmsp,BOSS_DR12,BOSS,BOTH",2,1,0,2


## Grab spec 

### Need to do igmspec and qpq separately -- think about a way to join?

### Grab on XSpectrum1D then the other, collate then splice

In [94]:
np.sum(spec_tbl['ibest'] >= 0)

176